In [32]:
import quantecon as qe
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import optimize
from scipy.interpolate import interp1d

### Endogenous Grid Method

Problem:

#### Standard VFI Form:

$$V_{n+1}(k_i , z_j) = \max_{k'} \left\{\dfrac{(z_jk_i^\alpha + (1-\delta)k_i - k')^{1-\gamma}}{1-\gamma} + \beta \mathbb{E}( V_n(k',z') | z_j)\right\}$$
$$\text{s.t. } k' \geq k_{min}$$
$$ln(z') = \rho ln(z) + \eta'$$

#### EGM Form:

$$V_{n+1}(k , z_j) = \max_{k_i'} \left\{\dfrac{(z_jk^\alpha + (1-\delta)k - k_i')^{1-\gamma}}{1-\gamma} + \beta \mathbb{E}( V_n(k_i',z') | z_j)\right\}$$
$$\text{s.t. } ln(z') = \rho ln(z) + \eta'$$

So in the EGM form, we are creating grid for:
- capital grid for tomorrow's values
- Today's states

To understand why this method is better than standard VFI, we need to look at the FOC that we have to solve in standard VFI and compare it the FOC of EGM:

- Standard VFI: $(z_jk_i^\alpha + (1-\delta)k_i - k')^{-\gamma} = \beta \mathbb{E}(V_k(k',z')|z_j)$

- EGM: $z_k k^\alpha + (1-\delta)k = [\beta \mathbb{E}(V_k(k_i',z')|z_j)]^{\frac{-1}{\gamma}}+ k_i'$ 

Yes, even in EGM we need to solve for k (and it is a non-linear equation) but it is much easier to solve the first one for k' because we don't need to interpolate $\mathbb{E}(V_k(k',z')|z_j)$ for each off grid k' because all the k' s are on the grid (we chose the tomorrow's capital grid). Another beautiful thing is that instead of constantly evaluating expected value for tomorrow as in Standard VFI FOC, in EGM, it is enough to compute it once.

Furthermore, we can even avoid solving this non-linear equation by carefully changing the state variable from the beginning of period to end of period total **cash on hand**

Define: $Y = z_jk^\alpha + (1-\delta)k$

Then Belmann equation becomes:

$$\hat V(Y,z_j) = \max_{k'} \left\{ \dfrac{(Y-k')^{1-\gamma}}{1-\gamma} + \underbrace{\beta \mathbb{E}(\hat V(Y',z')|z_j)}_{\mathbb{V}(k_i',z_j)}\right\}$$
$$\text{s.t. } Y' = z'k'^\alpha + (1-\delta)k'$$
$$ k' \geq k_{min}$$
$$ln(z') = \rho ln(z) + \eta'$$

More simply:

$$\hat V(Y,z_j) = \max_{k'} \left\{ \dfrac{(Y-k')^{1-\gamma}}{1-\gamma} + \mathbb{V}(k_i',z_j)\right\}$$
$$\text{s.t. } Y' = z'k'^\alpha + (1-\delta)k'$$
$$ k' \geq k_{min}$$
$$ln(z') = \rho ln(z) + \eta'$$

From FOC:

$$ c^*(k_i', z_j)^{-γ}  = \mathbb{V}_k(k_i', z_j)$$

From this equation, obtain $c^*$ and use it in the resource constraint to find $Y^*$:

$$Y^*(k_i',z_j) = c^*(k_i', z_j) + k_i'$$

And with this cash in hands today, we can find LHS of the value function as:

$$\hat V(Y^*(k_i',z_j),z_j) =\dfrac{(c^*(k_i',z_j))^{1-\gamma}}{1-\gamma} + \mathbb{V}(k_i',z_j) $$




In [50]:
# Define Parameters:

n_k = 500  # Grids for capitals
n_A = 15  # Markov States.
δ = 0.9   # Depreciation.
α = 0.7   # Capital Share.
ρ = 0.98  # Memory of income
σ = 0.01  # Volatility of income.
β = 0.98  # Discont factor.
θ = 1.5   # Expanding grid coefficient.
error = 10e-6 # Error tolerance.
max_iter = 1000

markov = qe.markov.approximation.rouwenhorst(n= n_A, ybar = 1-ρ, sigma=σ, rho= ρ)

Π = markov.P   
A = markov.state_values 

# Maximum sustainable Capital:

K_max = 2
K_min = 0.1

# Use and expanding grid:

Kp = K_min + (K_max - K_min) * (np.linspace(0, 1, n_k)**θ)


#Initial guess
V_0 = np.zeros((n_A,n_k))
for i in range(n_A):
    for j in range(n_k):
        V_0[i,j] = Kp[j]**(α) * A[i]

#Some necessary matrices for computation
V_k = np.zeros((n_A,n_k))
c_star = np.zeros((n_A,n_k))
Y_star = np.zeros((n_A,n_k))
value = np.zeros((n_A,n_k))
Realvalue = np.zeros((n_A,n_k))

Y_prime = np.zeros((n_A,n_k))
for i in range(n_A):
    for j in range(n_k):
        Y_prime[i,j] = A[i]* (Kp[j]** α)  + (1-δ)*Kp[j]
dist = 1000
iterasyon = 0

In [51]:
delta = 10e-4
while dist > error and iterasyon < max_iter:
    for i in range(n_A):
        V_spline = CubicSpline(Kp,V_0[i,:])
        c_star[i,:] = ((V_spline(Kp + delta) - V_spline(Kp - delta))/ (2*delta))**(-1/2)
    
    Y_star = c_star + Kp
    
    Values = -c_star **(-1) + V_0


    for i in range(n_A):
        Y_i = Y_star[i,:]
        V_i = Values[i,:]

        sort_index = np.argsort(Y_i)
        Y_i_sorted = Y_i[sort_index]
        V_i_sorted = V_i[sort_index]

        interpolation = CubicSpline(Y_i_sorted, V_i_sorted)

        Realvalue[i,:] = interpolation(Y_prime[i,:])

    V_1 = β * np.dot(Π, Realvalue)

    dist = np.max(np.abs(V_1 - V_0))
    iterasyon += 1
    print('iterasyon: ', iterasyon)
    print('dist: ', dist)

    V_0 = V_1.copy()

iterasyon:  1
dist:  3.106236591406194
iterasyon:  2
dist:  5.241312286192844
iterasyon:  3
dist:  7.609639765850915
iterasyon:  4
dist:  9.806345267720506
iterasyon:  5
dist:  11.376832138577246
iterasyon:  6
dist:  12.16563306805206
iterasyon:  7
dist:  12.278608664967187
iterasyon:  8
dist:  11.964474952487123
iterasyon:  9
dist:  11.460921189824774
iterasyon:  10
dist:  10.913739543045068
iterasyon:  11
dist:  10.386928659270438
iterasyon:  12
dist:  9.90121027257149
iterasyon:  13
dist:  9.459398616268658
iterasyon:  14
dist:  9.057992347751778
iterasyon:  15
dist:  8.691824331104158
iterasyon:  16
dist:  8.35578454765303
iterasyon:  17
dist:  8.045381369215846
iterasyon:  18
dist:  7.756855658870762
iterasyon:  19
dist:  7.487129284768002
iterasyon:  20
dist:  7.233703711202196
iterasyon:  21
dist:  6.994554264498731
iterasyon:  22
dist:  6.768035779242439
iterasyon:  23
dist:  6.552804118891544
iterasyon:  24
dist:  6.347752811941973
iterasyon:  25
dist:  6.151962862304288
itera

In [52]:
V_1

array([[-452.86450418, -452.803549  , -452.69246503, ..., -385.60431006,
        -385.56449732, -385.52478778],
       [-429.54491243, -429.4875901 , -429.38312745, ..., -366.55103463,
        -366.51394517, -366.47695216],
       [-408.14644879, -408.09243173, -407.99399328, ..., -349.02161048,
        -348.98698107, -348.95244199],
       ...,
       [-264.77563557, -264.74297028, -264.68344656, ..., -230.27635253,
        -230.25709955, -230.23789821],
       [-255.1773832 , -255.14609087, -255.08906937, ..., -222.2335462 ,
        -222.21524294, -222.19698889],
       [-246.15920903, -246.12919923, -246.0745151 , ..., -214.66441333,
        -214.64699176, -214.62961716]])